# SAÉ S2 : LABYRINTHES

# Millésime 2023

In [1]:
# encoding:utf-8
#    _     __  __   _    _______   _                  _   _
#   /_\   |  \/  | /_\  |_  / __| (_)_ _    _ __ _  _| |_| |_  ___ _ _
#  / _ \  | |\/| |/ _ \  / /| _|  | | ' \  | '_ \ || |  _| ' \/ _ \ ' \
# /_/ \_\ |_|  |_/_/ \_\/___|___| |_|_||_| | .__/\_, |\__|_||_\___/_||_|
#                                          |_|   |__/
# sae@butinfo:~$

In [72]:
from random import randint, choice

## 3 Implémentation

In [73]:
class Maze:
    """
    Classe Labyrinthe
    Représentation sous forme de graphe non-orienté
    dont chaque sommet est une cellule (un tuple (l,c))
    et dont la structure est représentée par un dictionnaire
      - clés : sommets
      - valeurs : ensemble des sommets voisins accessibles
    """
    def __init__(self, height: int, width: int, empty: bool = False) -> None:
        """
        Constructeur d'un labyrinthe de height cellules de haut 
        et de width cellules de large 
        Les voisinages sont initialisés à des ensembles vides
        Remarque : dans le labyrinthe créé, chaque cellule est complètement emmurée
        """
        self.height    = height
        self.width     = width
        self.neighbors = {(i,j): set() for i in range(height) for j in range (width)}
        if empty:
            for i in range(height):
                for j in range(width):
                    if i > 0:
                        self.neighbors[(i,j)].add((i-1,j))
                        self.neighbors[(i-1,j)].add((i,j))
                    if j > 0:
                        self.neighbors[(i,j)].add((i,j-1))
                        self.neighbors[(i,j-1)].add((i,j))
                    if i < height-1:
                        self.neighbors[(i,j)].add((i+1,j))
                        self.neighbors[(i+1,j)].add((i,j))
                    if j < width-1:
                        self.neighbors[(i,j)].add((i,j+1))
                        self.neighbors[(i,j+1)].add((i,j))
        
    def info(self) -> str:
        """
        Affichage des attributs d'un objet 'Maze' (fonction utile pour deboguer)
        Retour:
            chaîne (string): description textuelle des attributs de l'objet
        """
        txt = "**Informations sur le labyrinthe**\n"
        txt += f"- Dimensions de la grille : {self.height} x {self.width}\n"
        txt += "- Voisinages :\n"
        txt += str(self.neighbors)+"\n"
        valid = True
        for c1 in {(i, j) for i in range(self.height) for j in range(self.width)}:
            for c2 in self.neighbors[c1]:
                if c1 not in self.neighbors[c2]:
                    valid = False
                    break
            else:
                continue
            break
        txt += "- Structure cohérente\n" if valid else f"- Structure incohérente : {c1} X {c2}\n"
        return txt

    def __str__(self) -> str:
        """
        Représentation textuelle d'un objet Maze (en utilisant des caractères ascii)
        Retour:
             chaîne (str) : chaîne de caractères représentant le labyrinthe
        """
        txt = ""
        # Première ligne
        txt += "┏"
        for j in range(self.width-1):
            txt += "━━━┳"
        txt += "━━━┓\n"
        txt += "┃"
        for j in range(self.width-1):
            txt += "   ┃" if (0,j+1) not in self.neighbors[(0,j)] else "    "
        txt += "   ┃\n"
        # Lignes normales
        for i in range(self.height-1):
            txt += "┣"
            for j in range(self.width-1):
                txt += "━━━╋" if (i+1,j) not in self.neighbors[(i,j)] else "   ╋"
            txt += "━━━┫\n" if (i+1,self.width-1) not in self.neighbors[(i,self.width-1)] else "   ┫\n"
            txt += "┃"
            for j in range(self.width):
                txt += "   ┃" if (i+1,j+1) not in self.neighbors[(i+1,j)] else "    "
            txt += "\n"
        # Bas du tableau
        txt += "┗"
        for i in range(self.width-1):
            txt += "━━━┻"
        txt += "━━━┛\n"

        return txt
    
    def add_wall(self, c1: tuple, c2: tuple) -> None:
        """
        Ajoute un nouveau mur entre les cellules de coordonnées c1 et c2.
        """
        # Facultatif : on teste si les sommets sont bien dans le labyrinthe
        assert 0 <= c1[0] < self.height and \
            0 <= c1[1] < self.width and \
            0 <= c2[0] < self.height and \
            0 <= c2[1] < self.width, \
            f"Erreur lors de l'ajout d'un mur entre {c1} et {c2} : les coordonnées de sont pas compatibles avec les dimensions du labyrinthe"
        # Ajout du mur
        if c2 in self.neighbors[c1]:      # Si c2 est dans les voisines de c1
            self.neighbors[c1].remove(c2) # on le retire
        if c1 in self.neighbors[c2]:      # Si c3 est dans les voisines de c2
            self.neighbors[c2].remove(c1) # on le retire
    
    def remove_wall(self, c1: tuple, c2: tuple) -> None:
        """
        Efface un mur entre les cellules de coordonnées c1 et c2.
        """
        # Facultatif : on teste si les sommets sont bien dans le labyrinthe
        assert 0 <= c1[0] < self.height and \
            0 <= c1[1] < self.width and \
            0 <= c2[0] < self.height and \
            0 <= c2[1] < self.width, \
            f"Erreur lors de la suppression d'un mur entre {c1} et {c2} : les coordonnées de sont pas compatibles avec les dimensions du labyrinthe"
        # Suppression du mur
        if c2 not in self.neighbors[c1]:  # Si c2 n'est pas dans les voisines de c1
            self.neighbors[c1].add(c2)    # on l'ajoute
        if c1 not in self.neighbors[c2]:  # Si c3 n'est pas dans les voisines de c2
            self.neighbors[c2].add(c1)    # on l'ajoute
    
    def fill(self) -> None:
        """
        Ajoute tous les murs du labyrinthe.
        """
        for i in range(self.height):
            for j in range(self.width):
                if i > 0:
                    self.add_wall((i,j), (i-1,j))
                if j > 0:
                    self.add_wall((i,j), (i,j-1))
                if i < self.height-1:
                    self.add_wall((i,j), (i+1,j))
                if j < self.width-1:
                    self.add_wall((i,j), (i,j+1))
    
    def empty(self) -> None:
        """
        Efface tous les murs du labyrinthe.
        """
        for i in range(self.height):
            for j in range(self.width):
                if i > 0:
                    self.remove_wall((i,j), (i-1,j))
                if j > 0:
                    self.remove_wall((i,j), (i,j-1))
                if i < self.height-1:
                    self.remove_wall((i,j), (i+1,j))
                if j < self.width-1:
                    self.remove_wall((i,j), (i,j+1))
                    
    def get_walls(self) -> list:
        """
        Affichage de tous les murs du labyrinthe.
        Retour:
             liste (list) : liste représentant tous les murs du labyrinthe
        """
        murs = []
        for i in range(self.height):
            for j in range(self.width):
                voisins = []
                if j != self.height-1:
                    voisins += [(i, j+1)]
                if i != self.width-1:
                    voisins += [(i+1, j)]
                for k in range(len(voisins)):
                    if voisins[k] not in self.neighbors[(i, j)]:
                        murs.append(((i,j), voisins[k]))
        return murs
    
    def get_contiguous_cells(self, c1: tuple) -> list:
        """
        Affichage des murs contigus de la cellule c1 dans le labyrinthe.
        Retour:
             liste (list) : liste représentant les murs contigus de la cellule c1
        """
        voisins = []
        if c1[0] != self.width-1:
            voisins += [(c1[0]+1, c1[1])]
        if c1[1] > 0:
            voisins += [(c1[0], c1[1]-1)]
        if c1[1] != self.height-1:
            voisins += [(c1[0], c1[1]+1)]
        if c1[0] > 0:
            voisins += [(c1[0]-1, c1[1])]
        return voisins
    
    def get_reachable_cells(self, c1: tuple) -> list:
        """
        Affichage des murs accessibles de la cellule c1 dans le labyrinthe.
        Retour:
             liste (list) : liste représentant les murs accessibles de la cellule c1
        """
        accessibles = []
        voisins = self.get_contiguous_cells(c1)
        for i in range(len(voisins)):
            if voisins[i] in self.neighbors[c1]:
                        accessibles += [voisins[i]]
        return accessibles
    
    @classmethod
    def gen_btree(cls, h: int, w: int) -> list:
        """
        Représentation d'un labyrinthe par algorithme de génération par arbre binaire.
        Retour:
             liste (list) : liste représentant un dictionnaire de cellule et de cellule(s) voisine(s)
        """
        laby = Maze(h, w, empty = False)
        for i in range(h):
            for j in range(w):
                if (j != h-1) and (i == w-1):
                    laby.remove_wall((i, j), (i, j+1))
                elif (i != w-1) and (j == h-1):
                    laby.remove_wall((i, j), (i+1, j))
                elif (i != w-1) and (j != h-1):
                    hasard = randint(0,1)
                    if hasard == 0:
                        laby.remove_wall((i, j), (i, j+1))
                    else:
                        laby.remove_wall((i, j), (i+1, j))
        return laby
    
    @classmethod
    def gen_sidewinder(cls, h: int, w: int) -> list:
        """
        Représentation d'un labyrinthe par algorithme de génération sidewinder.
        Retour:
             liste (list) : liste représentant un dictionnaire de cellule et de cellule(s) voisine(s)
        """
        laby = Maze(h, w, empty = False)
        for i in range(h-1):
            sequence = []
            for j in range(w-1):
                sequence += [(i, j)]
                hasard = randint(0,1)
                if hasard == 0:
                    laby.remove_wall((i, j), (i, j+1))
                else:
                    cell1 = choice(sequence)
                    laby.remove_wall((cell1[0], cell1[1]), (cell1[0]+1, cell1[1]))
                    sequence = []
            sequence += [(i, w-1)]
            cell2 = choice(sequence)
            laby.remove_wall((cell2[0], cell2[1]), (cell2[0]+1, cell2[1]))
        for k in range(w-1):
            laby.remove_wall((h-1, k), (h-1, k+1))
        return laby
    
    

In [41]:
laby = Maze(4, 4)
print(laby.info())

**Informations sur le labyrinthe**
- Dimensions de la grille : 4 x 4
- Voisinages :
{(0, 0): set(), (0, 1): set(), (0, 2): set(), (0, 3): set(), (1, 0): set(), (1, 1): set(), (1, 2): set(), (1, 3): set(), (2, 0): set(), (2, 1): set(), (2, 2): set(), (2, 3): set(), (3, 0): set(), (3, 1): set(), (3, 2): set(), (3, 3): set()}
- Structure cohérente



In [42]:
print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛



In [43]:
laby.neighbors = {
    (0, 0): {(1, 0)},
    (0, 1): {(0, 2), (1, 1)},
    (0, 2): {(0, 1), (0, 3)},
    (0, 3): {(0, 2), (1, 3)},
    (1, 0): {(2, 0), (0, 0)},
    (1, 1): {(0, 1), (1, 2)},
    (1, 2): {(1, 1), (2, 2)},
    (1, 3): {(2, 3), (0, 3)},
    (2, 0): {(1, 0), (2, 1), (3, 0)},
    (2, 1): {(2, 0), (2, 2)},
    (2, 2): {(1, 2), (2, 1)},
    (2, 3): {(3, 3), (1, 3)},
    (3, 0): {(3, 1), (2, 0)},
    (3, 1): {(3, 2), (3, 0)},
    (3, 2): {(3, 1)},
    (3, 3): {(2, 3)}
}

In [44]:
print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃           ┃
┣   ╋   ╋━━━╋   ┫
┃   ┃       ┃   ┃
┣   ╋━━━╋   ╋   ┫
┃           ┃   ┃
┣   ╋━━━╋━━━╋   ┫
┃           ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛



In [45]:
laby.neighbors[(1, 3)].remove((2, 3))
laby.neighbors[(2, 3)].remove((1, 3))
print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃           ┃
┣   ╋   ╋━━━╋   ┫
┃   ┃       ┃   ┃
┣   ╋━━━╋   ╋━━━┫
┃           ┃   ┃
┣   ╋━━━╋━━━╋   ┫
┃           ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛



In [46]:
laby.neighbors[(1, 3)].add((2, 3))
laby.neighbors[(2, 3)].add((1, 3))
print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃           ┃
┣   ╋   ╋━━━╋   ┫
┃   ┃       ┃   ┃
┣   ╋━━━╋   ╋   ┫
┃           ┃   ┃
┣   ╋━━━╋━━━╋   ┫
┃           ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛



In [47]:
laby.neighbors[(1, 3)].remove((2, 3))
print(laby)
print(laby.info())

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃           ┃
┣   ╋   ╋━━━╋   ┫
┃   ┃       ┃   ┃
┣   ╋━━━╋   ╋━━━┫
┃           ┃   ┃
┣   ╋━━━╋━━━╋   ┫
┃           ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛

**Informations sur le labyrinthe**
- Dimensions de la grille : 4 x 4
- Voisinages :
{(0, 0): {(1, 0)}, (0, 1): {(1, 1), (0, 2)}, (0, 2): {(0, 1), (0, 3)}, (0, 3): {(0, 2), (1, 3)}, (1, 0): {(2, 0), (0, 0)}, (1, 1): {(0, 1), (1, 2)}, (1, 2): {(1, 1), (2, 2)}, (1, 3): {(0, 3)}, (2, 0): {(1, 0), (2, 1), (3, 0)}, (2, 1): {(2, 0), (2, 2)}, (2, 2): {(1, 2), (2, 1)}, (2, 3): {(3, 3), (1, 3)}, (3, 0): {(3, 1), (2, 0)}, (3, 1): {(3, 2), (3, 0)}, (3, 2): {(3, 1)}, (3, 3): {(2, 3)}}
- Structure incohérente : (2, 3) X (1, 3)



In [48]:
laby.neighbors[(2, 3)].remove((1,3))

In [49]:
c1 = (1, 3)
c2 = (2, 3)
if c1 in laby.neighbors[c2] and c2 in laby.neighbors[c1]:
    print(f"Il n'y a pas de mur entre {c1} et {c2} car elles sont mutuellement voisines")
elif c1 not in laby.neighbors[c2] and c2 not in laby.neighbors[c1]:
    print(f"Il y a un mur entre {c1} et {c2} car {c1} n'est pas dans le voisinage de {c2} et {c2} n'est pas dans le voisinage de {c1}")
else:
    print(f"Il y a une incohérence de réciprocité des voisinages de {c1} et {c2}")

Il y a un mur entre (1, 3) et (2, 3) car (1, 3) n'est pas dans le voisinage de (2, 3) et (2, 3) n'est pas dans le voisinage de (1, 3)


In [50]:
c1 = (1, 3)
c2 = (2, 3)
if c1 in laby.neighbors[c2] and c2 in laby.neighbors[c1]:
    print(f"{c1} est accessible depuis {c2} et vice-versa")
elif c1 not in laby.neighbors[c2] and c2 not in laby.neighbors[c1]:
    print(f"{c1} n'est pas accessible depuis {c2} et vice-versa")
else:
    print(f"Il y a une incohérence de réciprocité des voisinages de {c1} et {c2}")

(1, 3) n'est pas accessible depuis (2, 3) et vice-versa


In [51]:
L = []
for i in range(laby.height):
    for j in range(laby.width):
        L.append((i,j))
print(f"Liste des cellules : \n{L}")

Liste des cellules : 
[(0, 0), (0, 1), (0, 2), (0, 3), (1, 0), (1, 1), (1, 2), (1, 3), (2, 0), (2, 1), (2, 2), (2, 3), (3, 0), (3, 1), (3, 2), (3, 3)]


In [52]:
laby = Maze(4, 4, empty = True)
print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃               ┃
┣   ╋   ╋   ╋   ┫
┃               ┃
┣   ╋   ╋   ╋   ┫
┃               ┃
┣   ╋   ╋   ╋   ┫
┃               ┃
┗━━━┻━━━┻━━━┻━━━┛



In [53]:
laby = Maze(4, 4, empty = False)
print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛



## 4 Manipulation de labyrinthes

In [54]:
laby = Maze(5, 5, empty = True)
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛



In [55]:
laby.add_wall((0,0), (0,1))
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃               ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛



In [56]:
laby = Maze(5, 5, empty = True)
laby.fill()
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛



In [57]:
laby.remove_wall((0, 0), (0, 1))
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃       ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛



In [58]:
laby.empty()
laby.add_wall((0, 0), (0, 1))
laby.add_wall((0, 1), (1, 1))
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃               ┃
┣   ╋━━━╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛



In [59]:
print(laby.get_walls())

[((0, 0), (0, 1)), ((0, 1), (1, 1))]


In [60]:
print(laby.get_contiguous_cells((0,1)))

[(1, 1), (0, 0), (0, 2)]


In [61]:
print(laby.get_reachable_cells((0,1)))

[(0, 2)]


## 5 Génération

### 5.1 Arbre binaire

In [68]:
laby = Maze.gen_btree(4, 4)
print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃       ┃
┣   ╋   ╋━━━╋   ┫
┃   ┃       ┃   ┃
┣   ╋━━━╋   ╋   ┫
┃       ┃       ┃
┣━━━╋   ╋━━━╋   ┫
┃               ┃
┗━━━┻━━━┻━━━┻━━━┛



### 5.2 Sidewinder

In [78]:
laby = Maze.gen_sidewinder(4, 4)
print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃       ┃   ┃   ┃
┣   ╋━━━╋   ╋   ┫
┃   ┃   ┃   ┃   ┃
┣   ╋   ╋   ╋   ┫
┃   ┃   ┃       ┃
┣   ╋   ╋━━━╋   ┫
┃               ┃
┗━━━┻━━━┻━━━┻━━━┛

